# Imports

In [1]:
import torch
# import bitsandbytes as bnb
import datetime
from peft import (
    PeftModel,
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model
)

from transformers import (
    LlamaForCausalLM,
    LlamaTokenizer,
    BitsAndBytesConfig
)
import transformers
from datasets import load_dataset, Dataset 

import pandas as pd

/home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /home/matheusalb/anaconda3/envs/llm/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


------

# Functions

## Load Model and Tokenizer

In [2]:
def get_model(model_name):
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4_bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True
    )

    peft_config =  LoraConfig(
        lora_alpha=16, # 16
        lora_dropout=0.1, # 0.05
        bias="none",
        task_type="CAUSAL_LM",
        r=64, # 8
        target_modules=[
            "q_proj",
            "v_proj",
            # "k_proj",
            # "o_proj"
        ]
    )
    model = LlamaForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map={"": 0},
        # bnb_4bit_use_double_quant=True
    )
    
    model = prepare_model_for_kbit_training(model, True)
    model = get_peft_model(model, peft_config)
    
    model.config.use_cache = False
    
    return model

def get_tokenizer(model_name):
    tokenizer = LlamaTokenizer.from_pretrained(
        model_name
    )
    tokenizer.pad_token = (
        0
    )
    tokenizer.padding_side = "left"
    
    return tokenizer

def create_and_prepare_model(model_name):
    return get_model(model_name), get_tokenizer(model_name)

## Create Dataset

In [3]:

def get_dataset(path):
    instruction = 'Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante'
    messages = {}
    messages['instruction'] =[]
    messages['input'] = []
    messages['output'] = [] 
    
    df = pd.read_csv(path, skip_blank_lines=True)
    df = df.dropna(how='all')
    for _, linha in df.iterrows():
        messages['instruction'].append(instruction)
        messages['input'].append(linha['comentario'])
        messages['output'].append(linha['sugestaoResposta'])
            
    return Dataset.from_dict(messages)

## Create Prompt

In [4]:
def generate_prompt(data_point):
    prompt = f'''\
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{data_point['instruction']}
### Input:
{data_point['input']}
### Response:
{data_point['output']}
'''.strip()               
    return prompt

## Tokenize Prompts

In [5]:
def tokenize(prompt, max_lenght=300, add_eos_token=True):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=max_lenght,
        padding=False,
        return_tensors=None
    )
    
    ## adicionando eos token ao final dos tokens
    if (
        result['input_ids'][-1] != tokenizer.eos_token_id
        and len(result['input_ids']) < max_lenght
        and add_eos_token
    ):
        result['input_ids'].append(tokenizer.eos_token_id)
        result['attention_mask'].append(1)
        
    # generative models: labels are the same as the input
    result['labels'] = result['input_ids'].copy()
    
    return result

## Generate and Tokenize the prompts

In [6]:
def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_prompt = tokenize(full_prompt)
    return tokenized_prompt

------

# Run

## Defining Parameters

In [7]:
model_name = 'openlm-research/open_llama_7b_v2'
train_path = '../../../data/train_base.csv'
validation_path = '../../../data/validation_base.csv'    
OUTPUT_DIR = "./results/qlora_falcon7b_transformer_trainer"+datetime.datetime.now().isoformat()

training_arguments = transformers.TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=2, # micro_batch_size
    gradient_accumulation_steps=4, # batch_size / micro_batch_size
    # num_train_epochs=80,
    max_steps=100,
    save_steps=100,
    optim="paged_adamw_32bit", # adamw_torch
    save_total_limit=3,
    logging_steps=10,
    learning_rate=2e-4, # 3e-4
    fp16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03, #0.05
    # warmup_steps=100,
    lr_scheduler_type="constant", #cosine
    evaluation_strategy='steps',
    eval_steps=20,
    report_to="wandb"
)

## Loading Model and Tokenizer

In [8]:
model, tokenizer = create_and_prepare_model(model_name)
model.print_trainable_parameters()

Loading checkpoint shards: 100%|██████████| 2/2 [00:22<00:00, 11.24s/it]

trainable params: 33,554,432 || all params: 3,533,967,360 || trainable%: 0.9494833591219133


## Preprocessing the datasets

In [9]:
train_data = get_dataset(train_path)
validation_data = get_dataset(validation_path)

train_data = train_data.map(generate_and_tokenize_prompt)
validation_data = validation_data.map(generate_and_tokenize_prompt)

## Data Collator

In [10]:
# pad_to_multiple_of -> An integer representing the maximum sequence length, rounded up to the nearest multiple of this value.
data_collator = transformers.DataCollatorForSeq2Seq(
    tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
)


## Trainer

In [11]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=validation_data,
    args=training_arguments,
    data_collator=data_collator,
)

## Training

In [12]:
# old_state_dict = model.state_dict
# model.state_dict = (
#     lambda self, *_, **__: get_peft_model_state_dict(
#         self, old_state_dict()
#     )
# ).__get__(model, type(model))
# model = torch.compile(model)

In [13]:
trainer.train()
model.save_pretrained(OUTPUT_DIR)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: matheusalb. Use `wandb login --relogin` to force relogin


 10%|█         | 10/100 [00:46<06:37,  4.42s/it]

{'loss': 1.8737, 'learning_rate': 0.0002, 'epoch': 0.07}


 20%|██        | 20/100 [01:33<05:59,  4.50s/it]

{'loss': 1.3314, 'learning_rate': 0.0002, 'epoch': 0.15}


...

{'loss': 0.8333, 'learning_rate': 0.0002, 'epoch': 0.74}


                                                 
100%|██████████| 100/100 [08:50<00:00,  3.97s/it]

{'eval_loss': 0.9136545062065125, 'eval_runtime': 16.1668, 'eval_samples_per_second': 8.412, 'eval_steps_per_second': 1.052, 'epoch': 0.74}


100%|██████████| 100/100 [09:00<00:00,  5.40s/it]


{'train_runtime': 545.5375, 'train_samples_per_second': 1.466, 'train_steps_per_second': 0.183, 'train_loss': 1.0148065090179443, 'epoch': 0.74}


# Inferencia

In [15]:
def gen_input(comment):
    prompt = f'''Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
{comment}
### Response:
'''           
    return prompt

In [16]:
generation_config=model.generation_config
generation_config.max_new_tokens=300
# generation_config.temeperature=0.7
# generation_config.top_k=0.9
generation_config.num_return_sequences=1
# generation_config.pad_token_id=tokenizer.eos_token_id
# generation_config.eos_token_id=tokenizer.eos_token_id

def inference(model, text):
    enconded = tokenizer(text, return_tensors="pt")
    outputs = model.generate(
        input_ids=enconded.input_ids,
        attention_mask=enconded.attention_mask,
        generation_config=generation_config
    )
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))


In [17]:
comment = 'Alimentação caríssima para um péssimo atendimento.. super mal atendido'
inference(model, gen_input(comment))

/home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/transformers/generation/utils.py:1448: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Alimentação caríssima para um péssimo atendimento.. super mal atendido
### Response:
Olá, agradecemos por compartilhar sua experiência conosco. Lamentamos muito pela má impressão causada pelo atendimento insatisfatório que você recebeu. Vamos analisar o ocorrido para melhorar nossos serviços. Agradecemos por nos informar.


In [18]:
#treino
comment='Filé estava duro. Ficamos decepcionados.'
inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Filé estava duro. Ficamos decepcionados.
### Response:
Olá, agradecemos por compartilhar sua experiência conosco. Lamentamos que a comida não tenha atendido às suas expectativas. Vamos analisar o filé para garantir que isso não aconteça novamente. Agradecemos por nos informar.


In [19]:
comment='''A falta de vaga para cadeirantes e idosos é uma falha terrível nos dias de hoje, além de ser lei não tivemos nenhum apoio ou mesmo interesse por parte dos funcionários para ajudar na questão do cadeirante que estava nos acompanhado.
Não vou deixar de avaliar os outros pontos, no que tange ao atendimento interno e as delícias que são servidas. tudo de bom. 
Mas não vá com cadeirante que passará sufoco.'''
inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
A falta de vaga para cadeirantes e idosos é uma falha terrível nos dias de hoje, além de ser lei não tivemos nenhum apoio ou mesmo interesse por parte dos funcionários para ajudar na questão do cadeirante que estava nos acompanhado.
Não vou deixar de avaliar os outros pontos, no que tange ao atendimento interno e as delícias que são servidas. tudo de bom. 
Mas não vá com cadeirante que passará sufoco.
### Response:
Olá, agradecemos pelo seu feedback sobre a falta de vaga para cadeirantes e idosos em nosso restaurante. Lamentamos muito pela experiência negativa que você teve. Vamos analisar a situação para melhorar essa questão. Quanto ao atendimento interno, estamos sempre buscando melhorar nosso ser

In [20]:
comment = 'Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!'
inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!
### Response:
Olá, agradecemos por compartilhar sua experiência conosco. Lamentamos muito pela má impressão causada pelo atendimento demorado e pelo frango com gosto de peixe. Vamos analisar esses pontos para melhorar nossos serviços. Agradecemos por nos informar sobre esses detalhes. Esperamos ter a oportunidade de recebê-la novamente para oferecer uma experiência melhor.


In [21]:
comment = 'Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!'
inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!
### Response:
Olá, agradecemos por compartilhar sua experiência conosco. Lamentamos muito pela má impressão causada pelo atendimento demorado e pelo frango com gosto de peixe. Vamos analisar esses pontos para melhorar nossos serviços. Agradecemos por nos informar sobre esses detalhes. Esperamos ter a oportunidade de recebê-la novamente para oferecer uma experiência melhor.


In [22]:
comment = 'Lugar estava quente e, apesar de poucos usuários, a comida demorou bastante a ficar pronta.'
inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Lugar estava quente e, apesar de poucos usuários, a comida demorou bastante a ficar pronta.
### Response:
Olá, agradecemos por compartilhar sua experiência conosco. Lamentamos que a sua visita tenha sido difícil devido ao calor e à demora na preparação da comida. Vamos analisar esses pontos para melhorar nossos serviços. Esperamos ter a oportunidade de recebê-la novamente para proporcionar uma experiência mais satisfatória.


In [23]:
comment = 'Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lado de fora espera cerca de 40 min pra ser atendido.'
inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lado de fora espera cerca de 40 min pra ser atendido.
### Response:
Olá, agradecemos pelo seu feedback. Lamentamos que tenha tido uma experiência negativa em nosso restaurante. Vamos analisar o seu comentário para melhorar nossos serviços. Agradecemos por nos informar sobre a dificuldade de atendimento na área interna com a área externa. Vamos trabalhar para resolver esse problema. Esperamos ter a oportunidade de recebê-la novamente para uma experiência melhor.


In [24]:
comment = 'O pior yakisoba que eu já comi na vida! Vem numa caixa com a massa toda amassada e o molho é horrível! Tive que jogar fora! Parecia comida de cachorro! Nunca mais!'
inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
O pior yakisoba que eu já comi na vida! Vem numa caixa com a massa toda amassada e o molho é horrível! Tive que jogar fora! Parecia comida de cachorro! Nunca mais!
### Response:
Olá, agradecemos pelo seu feedback sobre a nossa yakisoba. Lamentamos muito pela má experiência que você teve com a nossa comida. Vamos analisar o ocorrido para melhorar nossos processos de preparo. Agradecemos por nos informar sobre o problema. Esperamos ter a oportunidade de recebê-la novamente para oferecer uma experiência melhor.


In [25]:
comment= '''Cheguei para tomar café as 7h da manhã, pouquíssimas pessoas ,pedi um omelete levou 23 minutos a essa altura meu café já havia acabado, e chegou uma pessoa depois de mim e o pedido dela que era muito mais que o meu simples omelete saiu primeiro.
Péssimo atendimento .
Já fui bem atendida por outros  funcionários  mais essa turma da manhã 3 pessoas nota 2.'''
inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Cheguei para tomar café as 7h da manhã, pouquíssimas pessoas ,pedi um omelete levou 23 minutos a essa altura meu café já havia acabado, e chegou uma pessoa depois de mim e o pedido dela que era muito mais que o meu simples omelete saiu primeiro.
Péssimo atendimento .
Já fui bem atendida por outros  funcionários  mais essa turma da manhã 3 pessoas nota 2.
### Response:
Olá, agradecemos por compartilhar sua experiência conosco. Lamentamos muito pela má atendimento que você recebeu. Vamos analisar o ocorrido para evitar que isso aconteça novamente. Agradecemos por nos informar sobre o omelete que levou muito tempo para ser preparado.


In [26]:
#treino
comment='Não recomendo! Não faz muito meu estilo cantar para quem não conheço, coisa que é impossível sem marcar antes. Sim, existem vários ambientes tocando estilos diferentes mas é necessário muita sorte para conseguir um lugar onde todos gostem do que está tocando. Sem falar que, se tratando do karaokê: ninguém é profissional por isso as performances são horríveis e a musica e letra são retiradas do YouTube, mas mesmo assim o espaço insiste em cobrar entrada! Para finalizar a comida, que é bem cara, veio fria e não gostei de nada que comi! Nem a bebida salvou minha noite, paguei um absurdo pela menor porção de caipirinha que já vi! Teria feito melhor na minha casa!'

inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Não recomendo! Não faz muito meu estilo cantar para quem não conheço, coisa que é impossível sem marcar antes. Sim, existem vários ambientes tocando estilos diferentes mas é necessário muita sorte para conseguir um lugar onde todos gostem do que está tocando. Sem falar que, se tratando do karaokê: ninguém é profissional por isso as performances são horríveis e a musica e letra são retiradas do YouTube, mas mesmo assim o espaço insiste em cobrar entrada! Para finalizar a comida, que é bem cara, veio fria e não gostei de nada que comi! Nem a bebida salvou minha noite, paguei um absurdo pela menor porção de caipirinha que já vi! Teria feito melhor na minha casa!
### Response:
Olá, agradeço por compartil

In [27]:
#treino
comment = '''Eu gosto muito de tomar café aos domingos nessa panificadora, porém não sei o que esta acontecendo, ela está ficando muito suja.
Gosto da parte de cima,mas quando chego as mesas estão todas sujas, com xícaras, resto de lanches e etc.
Acredito que não estão higienizando.
Outro detalhe a tapioca está vindo muito salgada.
Espero que melhore, para que possamos continuar frequentando.'''
inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Eu gosto muito de tomar café aos domingos nessa panificadora, porém não sei o que esta acontecendo, ela está ficando muito suja.
Gosto da parte de cima,mas quando chego as mesas estão todas sujas, com xícaras, resto de lanches e etc.
Acredito que não estão higienizando.
Outro detalhe a tapioca está vindo muito salgada.
Espero que melhore, para que possamos continuar frequentando.
### Response:
Olá, agradecemos pelo seu feedback sobre a sua experiência em nosso restaurante. Lamentamos muito pela falta de limpeza e pela salinidade da tapioca. Vamos analisar esses pontos para melhorar nossos serviços. Agradecemos por nos informar sobre esses detalhes e esperamos ter a oportunidade de recebê-la novamente

In [28]:
comment = '''Eu gostaria de deixar minha indignação quanto o atendimento de um garçom.
Não estou  reclamando do restaurante.
Estive no restaurante umas duas vezes, foi tudo muito bom.
Mas me decepcionei na minha ultima vez.
Foi atendida por um garçom mal educado e sem ética pois o mesmo alem de ser Grosso ficava olhando para as minhas pernas. Fiquei muito constrangida com a situação.
Almoçamos tomamos. Sucos e cervejas, mesmo porque estavamos em 6 pessoas.
No final pedimos a conta e o mesmo  trouxe duas contas uma com mais itens incluindo a refeiçao, outra somente com as bebidas,achei estranho porque tinham me cobrado normal das últimas vezes. Em fim.
Como um gestor de um restaurante contrata pessoas sem conhecer seu carater?
Tudo bem que a mão de obra de estrangeiros seja mais barata. Mas vocês teriam que pensar no bem estar dos clientes.
Descobri o nome do garçom.
CARLOS AMED. Ele é cubano.
Ia fazer uma denuncia por assedio mas pensei na empresa. Por favor selecione melhor seus colaboradores.
Muito indignada.'''

inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Eu gostaria de deixar minha indignação quanto o atendimento de um garçom.
Não estou  reclamando do restaurante.
Estive no restaurante umas duas vezes, foi tudo muito bom.
Mas me decepcionei na minha ultima vez.
Foi atendida por um garçom mal educado e sem ética pois o mesmo alem de ser Grosso ficava olhando para as minhas pernas. Fiquei muito constrangida com a situação.
Almoçamos tomamos. Sucos e cervejas, mesmo porque estavamos em 6 pessoas.
No final pedimos a conta e o mesmo  trouxe duas contas uma com mais itens incluindo a refeiçao, outra somente com as bebidas,achei estranho porque tinham me cobrado normal das últimas vezes. Em fim.
Como um gestor de um restaurante contrata pessoas sem conhecer

In [30]:
comment = "Fui no dia 10/12/21 às 11:00 o restaurante não tinha ninguém ainda, fui fazer um pedido para viagem. Depois de quase 30 minutos de espera percebo que tinham clientes que chegaram depois comendo...questionei um garçom pela demora do pedido foi averiguar e não deu nenhuma satisfação. Aí percebi depois que o pedido estava no balcão aguardando o favor de algum garçom pegar e olha que nem estava lotado o restaurante...desrespeito com o cliente e garçons mal treinados.. nunca mais nesse lugar."
inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Fui no dia 10/12/21 às 11:00 o restaurante não tinha ninguém ainda, fui fazer um pedido para viagem. Depois de quase 30 minutos de espera percebo que tinham clientes que chegaram depois comendo...questionei um garçom pela demora do pedido foi averiguar e não deu nenhuma satisfação. Aí percebi depois que o pedido estava no balcão aguardando o favor de algum garçom pegar e olha que nem estava lotado o restaurante...desrespeito com o cliente e garçons mal treinados.. nunca mais nesse lugar.
### Response:
Olá, agradeço por compartilhar sua experiência conosco. Lamentamos muito pela demora no atendimento e pelo desrespeito com o seu pedido. Vamos analisar o ocorrido para garantir que isso não aconteça nov

In [31]:
comment = "Só não dou zero porque não dá e porque a torta de frango estava boazinha. Priorizam quem tem aparência de ter mais dinheiro e os garçons não fazem questão alguma de atender as pessoas. Olharam na minha cara, passaram direto até que eu implorasse pra ser atendida. :) Não anotaram meu pedido e ainda queriam colocar a culpa em mim pelo esquecimento. Enfim, não volto!"
inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Só não dou zero porque não dá e porque a torta de frango estava boazinha. Priorizam quem tem aparência de ter mais dinheiro e os garçons não fazem questão alguma de atender as pessoas. Olharam na minha cara, passaram direto até que eu implorasse pra ser atendida. :) Não anotaram meu pedido e ainda queriam colocar a culpa em mim pelo esquecimento. Enfim, não volto!
### Response:
Olá, agradecemos por compartilhar sua experiência conosco. Lamentamos que a sua visita não tenha sido satisfatória. Pedimos desculpas pelo ocorrido com a torta de frango e pelo atendimento insatisfatório. Vamos analisar o ocorrido para melhorar nossos serviços. Agradecemos por nos informar sobre esses pontos negativos e espera

In [32]:
comment = "Pedimos um prato de bife ao molho madeira sendo que o feijão veio com gosto de molho madeira e o bife veio com gosto de queijo então eu recomendaria que vocês se dedicassem mais e as atende vistas são muito de humor baixo"

inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Pedimos um prato de bife ao molho madeira sendo que o feijão veio com gosto de molho madeira e o bife veio com gosto de queijo então eu recomendaria que vocês se dedicassem mais e as atende vistas são muito de humor baixo
### Response:
Olá, agradecemos por compartilhar sua experiência conosco. Lamentamos que você tenha tido uma má impressão com o prato de bife ao molho madeira. Vamos analisar o ocorrido para evitar que isso aconteça novamente. Quanto ao feijão, vamos revisar nossos processos para garantir que o gosto seja adequado. Agradecemos por nos informar sobre esses detalhes. Esperamos ter a oportunidade de recebê-la novamente para oferecer uma experiência melhor.


In [33]:
comment = "Pedi no ifood camarão empanado e mandaram camarão alho e óleo muito sem graça e poucas unidades. Fui inventar de experimentar por causa da propaganda no Instagram e me ferrei! Não recomendo."

inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Pedi no ifood camarão empanado e mandaram camarão alho e óleo muito sem graça e poucas unidades. Fui inventar de experimentar por causa da propaganda no Instagram e me ferrei! Não recomendo.
### Response:
Olá, agradecemos por compartilhar sua experiência conosco. Lamentamos muito pela má impressão causada pelo serviço que você recebeu. Vamos analisar o ocorrido para melhorar nossos processos. Agradecemos por nos informar sobre a falta de graça e quantidade em nosso camarão empanado. Esperamos ter a oportunidade de recebê-la novamente para oferecer uma experiência melhor.
